### Data exploring

In [1]:
import os
files = [f for f in os.listdir('.') if f.endswith('.csv')]

In [2]:
files

['employment.csv', 'personal_info.csv', 'update_status.csv', 'vehicles.csv']

In [3]:
import csv
import datetime
from itertools import islice

def preview_csv(fname):
    if not fname.endswith('.csv'):
        raise ValueError('Not a csv file')
    with open(fname, 'r') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in islice(csv_reader, 5):
            print(row)

In [4]:
for f in files:
    print(f'Preview of {f}')
    preview_csv(f)
    print('-'*45)

Preview of employment.csv
['employer', 'department', 'employee_id', 'ssn']
['Stiedemann-Bailey', 'Research and Development', '29-0890771', '100-53-9824']
['Nicolas and Sons', 'Sales', '41-6841359', '101-71-4702']
['Connelly Group', 'Research and Development', '98-7952860', '101-84-0356']
['Upton LLC', 'Marketing', '56-9817552', '104-22-0928']
---------------------------------------------
Preview of personal_info.csv
['ssn', 'first_name', 'last_name', 'gender', 'language']
['100-53-9824', 'Sebastiano', 'Tester', 'Male', 'Icelandic']
['101-71-4702', 'Cayla', 'MacDonagh', 'Female', 'Lao']
['101-84-0356', 'Nomi', 'Lipprose', 'Female', 'Yiddish']
['104-22-0928', 'Justinian', 'Kunzelmann', 'Male', 'Dhivehi']
---------------------------------------------
Preview of update_status.csv
['ssn', 'last_updated', 'created']
['100-53-9824', '2017-10-07T00:14:42Z', '2016-01-24T21:19:30Z']
['101-71-4702', '2017-01-23T11:23:17Z', '2016-01-27T04:32:57Z']
['101-84-0356', '2017-10-04T11:21:30Z', '2016-09-2

## Goal 1



Your first task is to create iterators for each of the four files that contained cleaned up data, of the correct type (e.g. string, int, date, etc), and represented by a named tuple.

For now these four iterators are just separate, independent iterators.

In [5]:
employer_col = ['STRING', 'STRING','INT','INT']
personal_info_col = ['INT', 'STRING', 'STRING', 'STRING','STRING', 'STRING']
update_status_col = ['INT','DATETIME','DATETIME']
vehicles_col = ['INT', 'STRING', 'STRING','INT']

file_col_types = [employer_col, personal_info_col, update_status_col, vehicles_col]

In [6]:
import datetime
from collections import namedtuple

def cast(data_type, value):
    if data_type == 'INT':
        value = value.replace('-','')
        return int(value)
    elif data_type =='DATETIME':
        
        return datetime.datetime.strptime(value, '%Y-%m-%dT%H:%M:%SZ')
    else:
        return str(value)
    
def cast_row(data_row, data_types):
    return [cast(data_type, value) 
            for data_type, value in zip(data_types, data_row)]


def make_iterator_namedtuple(fname, col_types:list):
    if not fname.endswith('.csv'):
        raise ValueError('Not a csv file')
        
    with open(fname, 'r') as csvfile:
        content = csv.reader(csvfile)
        ntuple = namedtuple(fname.split('.')[0], next(content))
        for row in content:
            row = cast_row(row, col_types)
            yield ntuple(*row)



In [7]:
for fname, col_type in zip(files, file_col_types):
    print(fname, col_type)

employment.csv ['STRING', 'STRING', 'INT', 'INT']
personal_info.csv ['INT', 'STRING', 'STRING', 'STRING', 'STRING', 'STRING']
update_status.csv ['INT', 'DATETIME', 'DATETIME']
vehicles.csv ['INT', 'STRING', 'STRING', 'INT']


In [8]:
employment_iter = make_iterator_namedtuple('employment.csv', employer_col)
personal_info_iter = make_iterator_namedtuple('personal_info.csv', personal_info_col)
update_status_iter = make_iterator_namedtuple('update_status.csv', update_status_col)
vehicles_iter = make_iterator_namedtuple('vehicles.csv', vehicles_col)

In [9]:
for _ in range(2):
    print(next(employment_iter))
    print(next(personal_info_iter))
    print(next(update_status_iter))
    print(next(vehicles_iter))
    print('-'*45)

employment(employer='Stiedemann-Bailey', department='Research and Development', employee_id=290890771, ssn=100539824)
personal_info(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
update_status(ssn=100539824, last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
vehicles(ssn=100539824, vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993)
---------------------------------------------
employment(employer='Nicolas and Sons', department='Sales', employee_id=416841359, ssn=101714702)
personal_info(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
update_status(ssn=101714702, last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
vehicles(ssn=101714702, vehicle_make='Ford', vehicle_model='Mustang', model_year=1997)
---------------------------------------------


In [10]:
preview_csv('employment.csv')

['employer', 'department', 'employee_id', 'ssn']
['Stiedemann-Bailey', 'Research and Development', '29-0890771', '100-53-9824']
['Nicolas and Sons', 'Sales', '41-6841359', '101-71-4702']
['Connelly Group', 'Research and Development', '98-7952860', '101-84-0356']
['Upton LLC', 'Marketing', '56-9817552', '104-22-0928']


### Goal 2:
Create a single iterable that combines all the columns from all the iterators.

The iterable should yield named tuples containing all the columns.
Make sure that the SSN's across the files match!

All the files are guaranteed to be in SSN sort order, and every SSN is unique, and every SSN appears in every file.

Make sure the SSN is not repeated 4 times - one time per row is enough!

In [11]:
class OpenFiles:
    def __init__(self, file_names, mode):
        self.file_names = file_names
        self.mode = mode
        self.file_objs = []

    def __enter__(self):
        print('opening files...')
        for fname in self.file_names:
            self.file_objs.append(open(fname, self.mode))
        return [csv.DictReader(fobj) for fobj in self.file_objs]          
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
#         print('closing file...')
        _ = [fobj.close() for fobj in self.file_objs]
        return False

In [12]:
def get_complete_details(files_list):
    from collections import ChainMap
    complete_details = []
    with OpenFiles(files_list, 'r') as csvdictreaders:
        headers = set([header for reader in csvdictreaders for header in reader.fieldnames])
        details = namedtuple('CompleteDetails', headers)
        # https://stackoverflow.com/questions/37584544/dict-merge-in-a-dict-comprehension
        try:
            while True:
                complete_details.append(details(**dict(ChainMap(*[next(c) for c in csvdictreaders]))))
        except:
            pass
    return complete_details

In [13]:
def get_merged_casted_details():
    employment_iter = make_iterator_namedtuple('employment.csv', employer_col)
    personal_info_iter = make_iterator_namedtuple('personal_info.csv', personal_info_col)
    update_status_iter = make_iterator_namedtuple('update_status.csv', update_status_col)
    vehicles_iter = make_iterator_namedtuple('vehicles.csv', vehicles_col)

    # duplicates 
    # headers = employer_col + personal_info_col + update_status_col + vehicles_col

    headers = ['ssn', 'first_name', 'last_name', 'gender', 'language','vehicle_make', 'vehicle_model', 'model_year',
         'employer', 'department', 'employee_id','last_updated', 'created']

    details = namedtuple('CompleteDetails', headers)

    try:
        while True:
            personal_info = next(personal_info_iter)
            ssn, *vehicle= next(vehicles_iter)
            *employer, ssn = next(employment_iter)
            ssn, *update = next(update_status_iter)
            yield details(*personal_info,*vehicle, *employer, *update)
    except:
        pass

In [14]:
merged_details = get_merged_casted_details()

In [15]:
for _ in range(4):
    print(next(merged_details))
    print('--'*20)

CompleteDetails(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993, employer='Stiedemann-Bailey', department='Research and Development', employee_id=290890771, last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
----------------------------------------
CompleteDetails(ssn=101714702, first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao', vehicle_make='Ford', vehicle_model='Mustang', model_year=1997, employer='Nicolas and Sons', department='Sales', employee_id=416841359, last_updated=datetime.datetime(2017, 1, 23, 11, 23, 17), created=datetime.datetime(2016, 1, 27, 4, 32, 57))
----------------------------------------
CompleteDetails(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005, employer='Connell

### Goal 3

Next, you want to identify any stale records, where stale simply means the record has not been updated since 3/1/2017 (e.g. last update date < 3/1/2017). Create an iterator that only contains current records (i.e. not stale) based on the `last_updated` field from the `status_update` file.

In [16]:
def get_current_records(date:"mm/dd/yyyy"):
    merged_details = get_merged_casted_details()
    date_obj = datetime.datetime.strptime(date,'%m/%d/%Y')
    for record in merged_details:
        if record.last_updated > date_obj:
            yield record

In [17]:
current_records = get_current_records('3/1/2017')

In [18]:
for _ in range(10):
    print(next(current_records))
    print('--'*20)

CompleteDetails(ssn=100539824, first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic', vehicle_make='Oldsmobile', vehicle_model='Bravada', model_year=1993, employer='Stiedemann-Bailey', department='Research and Development', employee_id=290890771, last_updated=datetime.datetime(2017, 10, 7, 0, 14, 42), created=datetime.datetime(2016, 1, 24, 21, 19, 30))
----------------------------------------
CompleteDetails(ssn=101840356, first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish', vehicle_make='GMC', vehicle_model='Yukon', model_year=2005, employer='Connelly Group', department='Research and Development', employee_id=987952860, last_updated=datetime.datetime(2017, 10, 4, 11, 21, 30), created=datetime.datetime(2016, 9, 21, 23, 4, 7))
----------------------------------------
CompleteDetails(ssn=104220928, first_name='Justinian', last_name='Kunzelmann', gender='Male', language='Dhivehi', vehicle_make='Oldsmobile', vehicle_model='Intrigue', mode

### Goal 4

Find the largest group of car makes for each gender.

Possibly more than one such group per gender exists (equal sizes).

In [19]:
from collections import Counter

In [20]:
male_groups = Counter()
female_groups = Counter()
merged_records = get_merged_casted_details()

for record in merged_records:
    if record.gender.lower() == 'male':
        male_groups[record.vehicle_make] += 1
    elif record.gender.lower() == 'female':
        female_groups[record.vehicle_make] += 1
    else:
        continue

In [21]:
male_groups.most_common(2)

[('Ford', 44), ('Chevrolet', 38)]

In [22]:
female_groups.most_common(5)

[('Ford', 48),
 ('Chevrolet', 48),
 ('Mitsubishi', 25),
 ('Toyota', 24),
 ('GMC', 23)]

In [23]:
male_groups, female_groups

(Counter({'Oldsmobile': 6,
          'Ford': 44,
          'GMC': 31,
          'Volkswagen': 19,
          'Aston Martin': 4,
          'Mercedes-Benz': 23,
          'Buick': 17,
          'Audi': 17,
          'Isuzu': 3,
          'Acura': 10,
          'Pontiac': 13,
          'Jaguar': 5,
          'Chevrolet': 38,
          'Dodge': 25,
          'Infiniti': 9,
          'Mitsubishi': 29,
          'Plymouth': 5,
          'Subaru': 8,
          'Lincoln': 7,
          'Cadillac': 11,
          'Toyota': 26,
          'Mazda': 14,
          'Volvo': 10,
          'Jeep': 7,
          'Maserati': 5,
          'Mercury': 12,
          'BMW': 12,
          'Hyundai': 10,
          'Chrysler': 3,
          'Lexus': 8,
          'Kia': 6,
          'Saturn': 3,
          'Maybach': 2,
          'Honda': 10,
          'Nissan': 7,
          'Scion': 1,
          'Bentley': 3,
          'Saab': 8,
          'Jensen': 1,
          'Smart': 1,
          'Porsche': 5,
          'Land Rove